# Facial Recognition 人物認識

![title](https://miro.medium.com/max/1028/1*MnbUSSXG1IDS9mXC0H8cKQ.png)

## 1. BACKGROUND ON FACE RECOGNITION

### 手順  
1. 顔検出  
2. 同じ顔データの学習  
3. 顔データの比較 
4. 顔認識  

## 2. THEORY OF OPENCV FACE RECOGNIZERS

1. Data Gathering: Gather face data (face images in this case) of the persons you want to identify.  
2. Train the Recognizer: Feed that face data and respective names of each face to the recognizer so that it can learn.  
3. Recognition: Feed new faces of that people and see if the face recognizer you just trained recognizes them.  

## 人物特定における3つの代表的アルゴリズム Eigenface, Fisherface, LBPH  
  
顔画像を固有顔に変換する、主成分分析（PCA）の処理過程は次のようになる  
  
1. 訓練用の画像（同じ照明条件、目や鼻の位置でスケーリング、同解像度）を準備  
2. 訓練用画像の平均を求め、平均画像を各画像から減算する  
3. 減算した画像の共分散行列を計算  
4. 共分散行列から、固有ベクトルと固有値を計算  
5. 主成分の選択    

### A) Eigenface Face Recognizer（固有顔認識）    
  
固有顔集合を生成するには、人間の顔のデジタル化された画像（同じ照明条件でなければならない）を多数集積し、目と鼻の位置を合わせる    


![title](https://www.superdatascience.com/wp-content/uploads/2017/08/Image-6-EigenFaces.png)  

### B) Fisherfaces Face Recognizer
  
fisherface は固有顔に比較して、照明や角度の違いに影響されにくいという特徴がある  

![title](https://camo.qiitausercontent.com/5528adfb9dc38e2c8d23e9389712415997927942/68747470733a2f2f71696974612d696d6167652d73746f72652e73332e616d617a6f6e6177732e636f6d2f302f3130373035362f31663761613562372d313966362d356661332d656664382d3138343563323938366363372e706e67)

### C) Local Binary Patterns Histogram (LBPH)  
  
顔を小さなセルに分割し、各エリアのヒストグラムを比較することで、前述の2つよりサンプルの顔のサイズや形が異なっていても精度よく検出できる

![title](https://camo.qiitausercontent.com/96213a8c9cce8aa1a42ca347d61905af07ff30bf/68747470733a2f2f71696974612d696d6167652d73746f72652e73332e616d617a6f6e6177732e636f6d2f302f3130373035362f61393165313362632d393332302d653236302d383932332d3430626665366233653638312e6a706567)

In [9]:
#モジュールのインポート
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [3]:
#there is no label 0 in our training data so subject name for index/label 0 is empty
subjects = ["", "Ramiz Raja", "Elvis Presley"]

#### OpenCVでは、各人物の顔画像にラベルをつける  
  
【人物】山田、田中、佐藤  
【画像】（A:山田1枚目、B:山田２枚目）、（C:田中1枚目、D:田中２枚目）、（E:佐藤１枚目、F:佐藤２枚目）  
【ラベル】(A,B) = 1, (C,D) = 2, (E,F) =3  

In [32]:
# まずは画像から顔だけを検出する関数を作る
def face_detection(img):
# グレースケール化    
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

In [ ]:
# LBPカスケード分類器を作成    
face_cascade = cv2.CascadeClassifier('lbpcascade_frontalface.xml')

# 画像内の顔を全て検出する
faces = face_cascade.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=5)

# 画像内に顔が一つも認識されない場合は元の画像が帰るようにする
if (len(faces) == 0):
    return None, None

# そもそも一つしか顔がない画像を使うため最初に認識された顔を検出する
(x, y, w, h) = faces[0]

# 顔と認識された部分の画像だけ切り抜いて出力する
return gray[y:y+w, x:x+h], faces[0]

In [30]:
# 各人物の学習用画像を読み込み、「顔画像」を検出する
# また、「顔画像」と「人物ラベル」の2つのリストを返す
def preproccesing_train_data(data_path): 
    
    directories = os.listdir(data_path)
    faces = []
    labels = []
    
    # 各ディレクトリ内の画像を読み込むループを作成
    for dir_name in directories:
        
        # 画像が入ったディレクトリだけを対象にする
        if not dir_name.startswith("s"):
            continue
            
        # ディレクトリの番号をラベル番号にする
        label = int(dir_name.replace("s", ""))
        
        # 画像が入ったフォルダにパスを通す
        subject_dir_path = data_path + "/" + dir_name
        
        # 画像のファイル名を取得する
        subject_images_names = os.listdir(subject_dir_path)
        
        # 各画像から顔を検出して、「顔画像」の切り抜きを"faces"リストに入れる
        for image_name in subject_images_names:
            
            if image_name.startswith("."):
                continue
            
            # 画像データへのパスを通す
            image_path = subject_dir_path + "/" + image_name
            
            # 人物画像の読み込み
            image = cv2.imread(image_path)
            
            cv2.imshow("Training on image...", image)
            cv2.waitKey(100)
            
            # 顔の検出（グレースケール）
            face, rectangle = face_detection(image)

            # 顔が検出できなかった画像のみリストに加える
            if face is not None:
                faces.append(face)
                labels.append(label)
    
    # openCV特有のスクリーン上での動きの指定
    cv2.destroyAllWindows()
    cv2.waitKey(1)
    cv2.destroyAllWindows()
    
    # 正しく顔を検出できた切り抜き画像のリストを回収する
    return faces, labels

#### ようやく学習用データを前処理する関数が完成したので実行する

In [ ]:
# サイズが全て等しい顔の切り抜き画像リストを出力する
print("Start loading data ...")
faces, labels = preproccesing_train_data('training-data')
print("All Done!")

# 前処理完了済みデータの概要を把握
print("Total number of faces : ", len(faces))
print("Total number of unique figures : ", len(faces))

ここではLBPHによる人物認識を試みる（一応他のアルゴリズムも準備する）

In [ ]:
# LBPH  
face_recognizer = cv2.face.createLBPHFaceRecognizer()
 
# EigenFace
#face_recognizer = cv2.face.createEigenFaceRecognizer()
 
# FisherFace
#face_recognizer = cv2.face.createFisherFaceRecognizer()

In [ ]:
# 学習用データを噛ませて学習開始
face_recognizer.train(faces, np.array(labels))

## Prediction 未学習のデータを読み込ませて人物判定を試みる